In [1]:
import datetime as dt
import requests
import random
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

import warnings

In [2]:
# URL to the data extraction site
url = "https://metsonline.dosm.gov.my/tradev2/product-coderesult"

# Obtain csrf_token from the website
csrf_token = 'dG5PZ09oay0nPiIMATBTeTsZHFAgKSJdRQQVKwRYJBQADRckHStGYA=='

# Create headers for all posts
headers = {
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8", 
"Cookie": "_ga_F8QG3XBL63=GS1.1.1679998862.1.1.1679998930.0.0.0; _ga_7Q47WMYPD5=GS1.1.1679998865.1.1.1679998930.0.0.0; _ga_HK905W412V=GS1.1.1679998865.1.1.1679998930.0.0.0; mp_f55a9b6a137ab21675ef8724f7864bde_mixpanel=%7B%22distinct_id%22%3A%20%2218727cb0e099da-0247f50f77e9aa-7a545471-144000-18727cb0e0a31%22%2C%22%24device_id%22%3A%20%2218727cb0e099da-0247f50f77e9aa-7a545471-144000-18727cb0e0a31%22%2C%22%24search_engine%22%3A%20%22google%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fwww.google.com%2F%22%2C%22%24initial_referring_domain%22%3A%20%22www.google.com%22%2C%22__timers%22%3A%20%7B%7D%7D; _ga_TF34V4E8EM=GS1.1.1679999831.1.1.1680004133.0.0.0; _ga_XSB7QCQ0M9=GS1.1.1681551362.3.1.1681551368.0.0.0; _ga=GA1.1.324035783.1679998837; PHPSESSID=3oi1at9dfj4ugn6nno2bsqdd60; _csrf=193a3c282709e8f8d32929d94372b30d2cc2a5402306edc5e12c04ca098a1819a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22SPmkNX8TOwS7oAIp1jZLK0O9tcXCRC-M%22%3B%7D; BNES_PHPSESSID=fk0GuBGEQJmmIMfAn9TJ6EK+FW/oxjgTr2AAO+vNTw1JHh1+YzlOkigmNmbMOtjxuyDVXvBqWx9L6ZHOVdyK0Unb9r8ZsuhgKRw0f3zlmA8=; BNES__csrf=T1JRVt/fspwmarDUSua9ANdq7/OTqhlhu8lprfzaWWaQtpaDzljVNyJUL1Y65guWawDI8LDmCHrD6nlIXud7o3ytcCBG5TDvi4Kd94SBmSCx1ED1HWM0YV+ztPXAQ5AhjCkcJExisSE5tRQSObfAv0WAFW/4frqxEg4pTyBtkcJZTi4Amki6jSChfqJAyShFuYOrg/ZKI0IsB8ShUR5UFcxqa3a28w/OAICYVhchalmoAAhmCMY22tozvta3gpBJGC+pONVCkNPozWC+7XVzDC5ZciTE34Uq"
}

In [3]:
# To obtain the targeted year with selected interval
start_year = 2000
current_year = dt.date.today().year
interval_year = 4

targeted_year = []

for year in range(start_year, current_year + 1, interval_year):
    if year + interval_year < 2023:
        ending_year = year + interval_year - 1
        targeted_year.append(ending_year)
    else:
        ending_year = current_year
        targeted_year.append(ending_year)
        
print(targeted_year)

[2003, 2007, 2011, 2015, 2019, 2023]


In [4]:
# To start extract data from url via request
data_raw = []
start = start_year

max_retries = 2

for year in targeted_year:
    payload_month = {
    '_csrf': csrf_token,
    'Tradev2[typeofsearch]': 'classification',
    'Tradev2[typedigit]': 7,
    'Tradev2[rangecode1]': 0,
    'Tradev2[rangecode2]': 9,
    # 'Tradev2[code_idcode]': ,
    # 'Tradev2[code_idcodedigit9]': ,
    # 'Tradev2[tradeflow]': ,
    'Tradev2[tradeflow][]': 'exports',
    # 'Tradev2[timeframe]': ,
    'Tradev2[timeframe]': 'month',
    # 'Tradev2[rangeyear]': ,
    # 'Tradev2[rangeyear2]': ,
    # 'Tradev2[rangeyearone]': ,
    # 'Tradev2[rangemonthone]': ,
    'Tradev2[mothdata]': start,
    'Tradev2[mothdata2]': year,
    # 'Tradev2[classification_serch]': ,
    # 'Tradev2[country2]': ,
    'Tradev2[geogroup]': 1,
    'Tradev2[geogroup]': 29,
    'Tradev2[codeshowby]': 'code'
    }
    
    for retry in range(max_retries):
        retry_delay = random.randint(10,15)  # seconds
        try:
            # Process the response here
            monthly_raw = requests.post(url, data=payload_month, headers=headers)
            data_raw.append(monthly_raw) # Store the extracted raw data into a list
            print(f"Data Extraction for {start}-{year} is Successful.")
            time.sleep(5)
            break # Break the retry loop if successful
            
        except (requests.ConnectionError) as e:
            print(f"Error: {e}. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            break  # Stop retrying if a different error occurs
    
    if year < 2023:
        start = year + 1
     
print(data_raw)

Data Extraction for 2000-2003 is Successful.
Data Extraction for 2004-2007 is Successful.
Data Extraction for 2008-2011 is Successful.
Data Extraction for 2012-2015 is Successful.
Data Extraction for 2016-2019 is Successful.
Data Extraction for 2020-2023 is Successful.
[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]


In [5]:
# Create an empty list for storing the raw text from data extraction
data_extracted = []

# To extract only the table value
for raw in data_raw:
    result = BeautifulSoup(raw.text, 'html.parser') # Parse the HTML
    table = result.find('table', class_='table-bordered') # Look up for the table
    
    # Extract table rows
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all(['th', 'td'])
        cols = [col.get_text(strip=True) for col in cols]
        data_extracted.append(cols)

In [6]:
# To loop through the sublist in the data_extracted, check for empty sublist, if a sublsit is found empty
# Do not append into the filtered_list
filtered_list = [sublist for sublist in data_extracted if sublist]


division_length = 12
# Using a for loop and iterates over a range of indices starting from 0 and an increamental value = division_length
divided_list = [filtered_list[i:i + division_length] for i in range(0, len(filtered_list), division_length)]

In [7]:
# Create an empty dataframe
data = pd.DataFrame()

for item in divided_list:
    transposed_data = list(map(list, zip(*item)))
    df = pd.DataFrame(transposed_data[1:], columns=transposed_data[0])
    
    df = df.drop('GRAND TOTAL', axis=1) # Remove GrandTotal Column  
    columns_label = df.iloc[1] 
    columns_label = columns_label.to_list()
    df = df.drop(index=[0,1], axis=0)
    
    # To concatanete dataframes into one
    data = pd.concat([data, df], ignore_index=True)

# Rename the dataframe column
columns_label[0] = 'YEAR'
data.columns = columns_label

display(data.head(13))

,YEAR,FOOD,BEVERAGES AND TOBACCO,"CRUDE MATERIALS, INEDIBLE","MINERAL FUELS, LUBRICANTS, ETC.",ANIMAL AND VEGETABLE OILS AND FATS,CHEMICALS,MANUFACTURED GOODS,MACHINERY & TRANSPORT EQUIPMENT,MISCELLANEOUS MANUFACTURED ARTICLES,MISCELLANEOUS TRANSACTIONS AND COMMODITIES
0,Jan 2000,"491,864,874","92,332,416","863,994,681","2,843,136,683","1,116,946,684","880,219,925","1,927,880,545","14,958,386,186","2,026,711,654","229,883,181"
1,Feb 2000,"425,641,394","77,214,349","689,603,279","2,881,747,034","1,099,756,943","1,072,302,224","1,910,675,351","16,986,581,083","1,922,449,106","217,203,495"
2,Mar 2000,"553,169,207","100,950,924","901,427,068","3,188,319,069","1,254,551,970","1,086,130,053","2,133,213,423","20,139,482,974","2,368,704,586","262,696,514"
3,Apr 2000,"509,671,983","98,515,223","897,853,450","2,455,441,551","1,116,557,420","1,087,280,719","2,174,508,072","17,989,050,119","2,333,513,182","240,322,014"
4,May 2000,"553,327,793","111,185,482","909,523,666","2,651,869,264","1,008,684,467","1,231,347,129","2,291,918,346","18,776,956,887","2,480,702,004","253,977,343"
5,Jun 2000,"574,066,614","112,047,681","923,797,559","2,885,169,740","1,100,698,246","1,376,535,977","2,191,009,429","19,830,751,054","2,505,145,689","263,364,009"
6,Jul 2000,"551,376,941","88,254,854","968,187,353","3,117,527,228","1,032,325,817","1,418,732,775","2,236,784,776","20,301,875,068","2,722,661,037","267,044,431"
7,Aug 2000,"564,107,493","101,873,018","848,391,883","2,946,768,259","929,934,519","1,290,185,907","2,273,679,985","21,969,621,903","2,809,204,982","273,941,568"
8,Sep 2000,"579,122,481","99,952,862","872,988,719","3,206,132,395","1,155,303,802","1,379,033,062","2,197,408,175","22,289,507,276","2,907,182,376","254,519,214"
9,Oct 2000,"541,286,968","115,090,581","822,098,600","3,373,358,792","1,123,505,049","1,202,689,184","2,343,395,279","20,403,990,329","2,755,527,683","290,897,109"


In [10]:
# Data Cleaning Part
# Remove rows that contain the total value of a year
clean_data = data[data['YEAR'].apply(lambda x: '-' not in x)]

column_names = clean_data.columns.to_list()
del column_names[0]

for col in column_names:
    clean_data[col] = clean_data[col].astype(str).str.replace(',','').astype(np.int64)
    
display(clean_data.head(20))
print(clean_data.dtypes)

C:\Users\Albert\AppData\Local\Temp\ipykernel_21512\1239355368.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data[col] = clean_data[col].astype(str).str.replace(',','').astype(np.int64)
C:\Users\Albert\AppData\Local\Temp\ipykernel_21512\1239355368.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data[col] = clean_data[col].astype(str).str.replace(',','').astype(np.int64)
C:\Users\Albert\AppData\Local\Temp\ipykernel_21512\1239355368.py:9: SettingWithCopyWarning: 
A value is trying to be s

,YEAR,FOOD,BEVERAGES AND TOBACCO,"CRUDE MATERIALS, INEDIBLE","MINERAL FUELS, LUBRICANTS, ETC.",ANIMAL AND VEGETABLE OILS AND FATS,CHEMICALS,MANUFACTURED GOODS,MACHINERY & TRANSPORT EQUIPMENT,MISCELLANEOUS MANUFACTURED ARTICLES,MISCELLANEOUS TRANSACTIONS AND COMMODITIES
0,Jan 2000,491864874,92332416,863994681,2843136683,1116946684,880219925,1927880545,14958386186,2026711654,229883181
1,Feb 2000,425641394,77214349,689603279,2881747034,1099756943,1072302224,1910675351,16986581083,1922449106,217203495
2,Mar 2000,553169207,100950924,901427068,3188319069,1254551970,1086130053,2133213423,20139482974,2368704586,262696514
3,Apr 2000,509671983,98515223,897853450,2455441551,1116557420,1087280719,2174508072,17989050119,2333513182,240322014
4,May 2000,553327793,111185482,909523666,2651869264,1008684467,1231347129,2291918346,18776956887,2480702004,253977343
5,Jun 2000,574066614,112047681,923797559,2885169740,1100698246,1376535977,2191009429,19830751054,2505145689,263364009
6,Jul 2000,551376941,88254854,968187353,3117527228,1032325817,1418732775,2236784776,20301875068,2722661037,267044431
7,Aug 2000,564107493,101873018,848391883,2946768259,929934519,1290185907,2273679985,21969621903,2809204982,273941568
8,Sep 2000,579122481,99952862,872988719,3206132395,1155303802,1379033062,2197408175,22289507276,2907182376,254519214
9,Oct 2000,541286968,115090581,822098600,3373358792,1123505049,1202689184,2343395279,20403990329,2755527683,290897109


YEAR                                          object
FOOD                                           int64
BEVERAGES AND TOBACCO                          int64
CRUDE MATERIALS, INEDIBLE                      int64
MINERAL FUELS, LUBRICANTS, ETC.                int64
ANIMAL AND VEGETABLE OILS AND FATS             int64
CHEMICALS                                      int64
MANUFACTURED GOODS                             int64
MACHINERY & TRANSPORT EQUIPMENT                int64
MISCELLANEOUS MANUFACTURED ARTICLES            int64
MISCELLANEOUS TRANSACTIONS AND COMMODITIES     int64
dtype: object


In [ ]:
# Save the dataframe into .
df.to_csv('filename.csv', index=False)